In [ ]:
import pandas as pd

In [ ]:
%run ../tasking_manager_stats/data_management

In [ ]:
df = pd.read_csv(os.path.join(get_data_dir(), 'merged_stats.csv'))
df['date'] = df['Year'].astype(str) + '-' + df['Month'].astype(str) + '-' + df['Day'].astype(str)
df['date'] = pd.to_datetime(df['date'], yearfirst=True)
df.head()

In [ ]:
df2 = df[(df['Hour'] > 17) & (df['Hour'] < 22)]
df2.head()

In [ ]:
mapathons = pd.read_csv(os.path.join('..', 'data', 'Mapathons_2020_02_08.csv'))
mapathons['Date'] = pd.to_datetime(mapathons['Date'], dayfirst=True)
mapathons.head()

In [ ]:
mapathons2 = pd.DataFrame()
for _, row in mapathons.iterrows():
    # Extract projects of the mapathon
    tasks = row['Tasks']
    if pd.isnull(tasks):
        continue
    projects = set()
    for s in tasks.split('/'):
        try:
            projects.add(int(s))
        except:
            pass
    for s in tasks.split(', '):
        try:
            projects.add(int(s))
        except:
            pass
    # Create new mapathon line for each project
    for project in projects:
        mapathons2 = pd.concat([mapathons2, pd.DataFrame(data=[(row['Date'], row['City'], project)], columns=['date', 'City', 'Project'])], axis=0, ignore_index=True)

In [ ]:
mapathons2.head()

In [ ]:
len(df2)

In [ ]:
len(mapathons2)

In [ ]:
df3 = pd.merge(df2.loc[df2['Type'] == 'MAPPING'], mapathons2, on=['date', 'Project'])

In [ ]:
len(df3)

In [ ]:
df3.head()

In [ ]:
df4 = df3[['date', 'Author', 'City', 'Project']].drop_duplicates()
df4.head()

In [ ]:
len(df4)

In [ ]:
tmp = df4[['Author', 'City']].drop_duplicates()
tmp.loc[tmp['Author'] == 'NicolasGrosjean']

In [ ]:
df4.loc[df4['Author'] == 'NicolasGrosjean']

In [ ]:
df5 = df4[['Author', 'date']].drop_duplicates().groupby('Author').count().date.reset_index()

In [ ]:
df5.loc[df5['Author'] == 'NicolasGrosjean']

# Validation

- WARNING : Divide validation time by simultanous validation tasks
- Own mapping and validation time
- Validation time on own mapping tasks

In [ ]:
df_project = pd.read_csv(os.path.join(get_data_dir(), 'stats', '6055.csv'))
df_project = df_project[df_project['Type'] == 'VALIDATION']
df_project.head()

In [ ]:
key = ['Year', 'Month', 'Day', 'Rel. Day', 'Hour', 'Minute', 'Second', 'Duration', 'Author', 'Type']
df_project2 = df_project.groupby(key).count().Task
df_project2 = df_project2.reset_index()
df_project2.tail()

In [ ]:
df_project3 = pd.merge(df_project, df_project2, on=key)
df_project3.tail()

In [ ]:
df_project3['Duration'] /= df_project3['Task_y']

In [ ]:
df_project3['Duration'].sum()

In [ ]:
df_project['Duration'].sum() - 18*336

In [ ]:
def compute_validation_time_by_task(project_id):
    df_project = pd.read_csv(os.path.join(get_data_dir(), 'stats', str(project_id) + '.csv'))
    df_project = df_project[df_project['Type'] == 'VALIDATION']
    key = ['Year', 'Month', 'Day', 'Rel. Day', 'Hour', 'Minute', 'Second', 'Duration', 'Author', 'Type']
    df_project2 = df_project.groupby(key).count().Task
    df_project2 = df_project2.reset_index()
    df_project3 = pd.merge(df_project, df_project2, on=key)
    df_project3['Duration'] /= df_project3['Task_y']
    df_project3 = df_project3[['Project', 'Task_x', 'Duration']]
    df_project3.columns = ['Project', 'Task', 'Duration']
    df_project3.groupby(['Project', 'Task']).sum().reset_index()
    return df_project3

In [ ]:
compute_validation_time_by_task(6055)['Duration'].sum()

In [ ]:
df_one_author = pd.read_csv(os.path.join(get_data_dir(), 'merged_stats_one_author_by_task_type.csv'))
df_one_author.head()

In [ ]:
df.head()

In [ ]:
df.loc[df.groupby(['Type', 'Task']).idxmax()['Duration']].head()